In [2]:
import pandas as pd


In [3]:
df = pd.read_csv('../datasets/dt_target.csv',sep=',')
df.head()

,product_id,customer_id,periodo,periodo_producto,nacimiento_producto,muerte_cliente,tn,cust_request_tn,cust_request_qty,cat1,cat2,cat3,brand,sku_size,stock_final,plan_precios_cuidados,periodo_dt,target
0,20524,10234,201701,6.48085,201701,201912,0.05300,0.05300,2.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.0,2017-01-01,-0.03786
1,20524,10234,201702,3.99755,201701,201912,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-01,0.00000
2,20524,10234,201703,7.14711,201701,201912,0.01514,0.01514,1.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.0,2017-03-01,-0.01514
3,20524,10234,201704,6.82163,201701,201912,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-04-01,0.03786
4,20524,10234,201705,9.25949,201701,201912,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-01,0.00000


In [4]:
df.shape

(16998193, 18)

In [5]:
# Ordenar por producto, cliente y tiempo
df["periodo_dt"] = pd.to_datetime(df["periodo_dt"], format='%Y-%m-%d')
df = df.sort_values(['product_id', 'customer_id', 'periodo_dt'])


df["month"] = df["periodo_dt"].dt.month
df["year"] = df["periodo_dt"].dt.year
df['quarter'] = df['month'].apply(lambda x: (x-1)//3 + 1)
df['semester'] = df['month'].apply(lambda x: 1 if x <=6 else 2)
df['is_month_end'] = df['month'].isin([1, 3, 5, 7, 8, 10, 12])  # Meses con 31 días
df['season'] = df['month']%12 // 3 + 1  # 1:Invierno, 2:Primavera, etc.
df['size_vs_category'] = df['sku_size'] / df.groupby('cat3')['sku_size'].transform('mean')


# Crear lags
df['lag_1m'] = df.groupby(['product_id', 'customer_id'])['tn'].shift(1)
df['lag_2m'] = df.groupby(['product_id', 'customer_id'])['tn'].shift(2)
df['lag_3m'] = df.groupby(['product_id', 'customer_id'])['tn'].shift(3)
df['lag_11m'] = df.groupby(['product_id', 'customer_id'])['tn'].shift(11)

# Promedio móvil
df['rolling_3m_mean'] = df.groupby(['product_id', 'customer_id'])['tn'].transform(
    lambda x: x.rolling(3, min_periods=1).mean())

df['rolling_6m_mean'] = df.groupby(['product_id', 'customer_id'])['tn'].transform(
    lambda x: x.rolling(6, min_periods=1).mean())

df['rolling_12m_mean'] = df.groupby(['product_id', 'customer_id'])['tn'].transform(
    lambda x: x.rolling(12, min_periods=1).mean())


# Tendencia anual
df['annual_trend'] = df.groupby(['product_id', 'month'])['tn'].transform('mean')

# Variación estacional
df['seasonal_variation'] = df['tn'] / df['annual_trend']

# segunda tanda de features

# Desviación estándar y coeficiente de variación
df['std_6m'] = df.groupby(['product_id', 'customer_id'])['tn'].transform(
    lambda x: x.shift(1).rolling(6, min_periods=2).std())

df['mean_6m'] = df['rolling_6m_mean']
df['cv_6m'] = df['std_6m'] / (df['mean_6m'] + 1e-5)  # Coef. de variación: std/mean


# Periodos desde última compra
df['comprado'] = (df['tn'] > 0).astype(int)
# df['periodos_desde_ultima_compra'] = df.groupby(['product_id', 'customer_id'])['comprado'].apply(
#     lambda x: x[::-1].cumsum()[::-1].where(x==1).ffill().fillna(0))
df['periodos_desde_ultima_compra'] = df.groupby(['product_id', 'customer_id'])['comprado'].transform(
    lambda x: x[::-1].cumsum()[::-1].where(x==1).ffill().fillna(0))
# Cantidad de meses con compra en últimos N meses
for window in [3, 6, 12]:
    df[f'freq_compra_{window}m'] = df.groupby(['product_id', 'customer_id'])['comprado'].transform(
        lambda x: x.shift(1).rolling(window, min_periods=1).sum())
    
from scipy.stats import mode

# Modo de cantidad (más común en últimos 6 meses)
def rolling_mode(x):
    return x.shift(1).rolling(6, min_periods=1).apply(lambda x: mode(x, keepdims=True)[0][0], raw=False)
#Moda o patrón de cantidad
df['modo_6m'] = df.groupby(['product_id', 'customer_id'])['tn'].transform(rolling_mode)
df['modo_diff'] = df['tn'] - df['modo_6m']
#Tendencia reciente
df['trend_3m'] = df.groupby(['product_id', 'customer_id'])['tn'].transform(
    lambda x: x.shift(1).rolling(3).apply(lambda y: y.iloc[-1] - y.iloc[0] if len(y) == 3 else 0))
#Promedio histórico total
df['media_historica_cliente_producto'] = df.groupby(['product_id', 'customer_id'])['tn'].transform(
    lambda x: x.expanding().mean())
#Ratio de compra cliente vs. total producto
df['participacion_producto'] = df['tn'] / (df['periodo_producto'] + 1e-5)
#Uso de stock
df['stock_vs_venta'] = df['stock_final'] / (df['tn'] + 1e-5)

df['meses_desde_nacimiento'] = df['periodo'] - df['nacimiento_producto']
df['meses_hasta_muerte_cliente'] = df['muerte_cliente'] - df['periodo']
df['productos_distintos_cliente_mes'] = df.groupby(['customer_id', 'periodo'])['product_id'].transform('nunique')
df['total_cliente_mes'] = df.groupby(['customer_id', 'periodo'])['tn'].transform('sum')
df['proporcion_producto_en_total_mes'] = df['tn'] / (df['total_cliente_mes'] + 1e-5)
# tn vs. cust_request_tn → indicador de si le entregaron lo que pidió
df['ratio_entregado_sobre_pedido'] = df['tn'] / (df['cust_request_tn'] + 1e-5)
# tn vs. sku_size → toneladas por unidad
df['tn_por_unidad'] = df['tn'] / (df['sku_size'] + 1e-5)

In [6]:
df.to_csv("../datasets/dt_fe2.csv", index=False, sep=",")

In [7]:
df.dtypes

product_id                                   int64
customer_id                                  int64
periodo                                      int64
periodo_producto                           float64
nacimiento_producto                          int64
muerte_cliente                               int64
tn                                         float64
cust_request_tn                            float64
cust_request_qty                           float64
cat1                                        object
cat2                                        object
cat3                                        object
brand                                       object
sku_size                                   float64
stock_final                                float64
plan_precios_cuidados                      float64
periodo_dt                          datetime64[ns]
target                                     float64
month                                        int32
year                           

In [8]:
df.describe()

,product_id,customer_id,periodo,periodo_producto,nacimiento_producto,muerte_cliente,tn,cust_request_tn,cust_request_qty,sku_size,...,media_historica_cliente_producto,participacion_producto,stock_vs_venta,meses_desde_nacimiento,meses_hasta_muerte_cliente,productos_distintos_cliente_mes,total_cliente_mes,proporcion_producto_en_total_mes,ratio_entregado_sobre_pedido,tn_por_unidad
count,1.699819e+07,1.699819e+07,1.699819e+07,1.699819e+07,1.699819e+07,1.699819e+07,1.699819e+07,2.940634e+06,2.940634e+06,2.933187e+06,...,1.699819e+07,1.699819e+07,1.126334e+06,1.699819e+07,1.699819e+07,1.699819e+07,1.699819e+07,1.699819e+07,2.940634e+06,2.933187e+06
mean,2.053031e+04,1.028305e+04,2.018053e+05,4.523226e+01,2.017144e+05,2.019064e+05,8.204286e-02,4.844765e-01,2.181631e+00,4.467299e+02,...,8.818597e-02,1.830970e-03,8.444327e+02,9.086700e+01,1.011014e+02,8.644326e+02,7.063342e+01,9.701238e-04,9.975881e-01,3.468941e-03
min,2.000100e+04,1.000100e+04,2.017010e+05,4.400000e-04,2.017010e+05,2.017010e+05,0.000000e+00,1.000000e-04,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,-4.584258e+03,0.000000e+00,0.000000e+00,7.810000e+02,0.000000e+00,0.000000e+00,3.785145e-04,9.100000e-07
25%,2.023500e+04,1.013800e+04,2.017090e+05,2.356280e+00,2.017010e+05,2.019120e+05,0.000000e+00,1.040000e-02,1.000000e+00,9.000000e+01,...,0.000000e+00,0.000000e+00,2.862348e+01,6.000000e+00,8.000000e+00,8.470000e+02,6.813700e-01,0.000000e+00,9.989837e-01,5.507547e-05
50%,2.048800e+04,1.027600e+04,2.018060e+05,9.997160e+00,2.017010e+05,2.019120e+05,0.000000e+00,4.013000e-02,1.000000e+00,2.400000e+02,...,1.257692e-04,0.000000e+00,1.891654e+02,1.020000e+02,1.040000e+02,8.560000e+02,6.520700e+00,0.000000e+00,9.997394e-01,1.965600e-04
75%,2.080500e+04,1.042200e+04,2.019030e+05,3.080259e+01,2.017010e+05,2.019120e+05,0.000000e+00,1.627500e-01,2.000000e+00,4.500000e+02,...,8.088889e-03,0.000000e+00,7.734967e+02,1.940000e+02,2.010000e+02,8.970000e+02,3.153658e+01,0.000000e+00,9.999360e-01,8.057067e-04
max,2.129900e+04,1.063700e+04,2.019120e+05,2.424467e+03,2.019090e+05,2.019120e+05,5.478785e+02,5.515614e+02,1.420000e+02,1.000000e+04,...,2.437955e+02,9.999995e-01,2.453337e+05,2.110000e+02,2.110000e+02,9.320000e+02,4.704263e+03,9.999987e-01,1.139297e+01,2.051552e+01
std,3.450224e+02,1.692170e+02,8.145169e+01,1.213034e+02,4.211088e+01,2.682285e+01,1.398516e+00,3.458918e+00,3.686673e+00,7.445894e+02,...,1.151480e+00,1.335544e-02,2.240954e+03,7.893427e+01,8.077464e+01,4.270546e+01,2.681095e+02,9.563166e-03,2.702053e-02,5.697891e-02


In [ ]:
def optimize_memory_usage(df, verbose=True, convert_bool=True):
    """
    Optimiza el uso de memoria de un DataFrame reduciendo tipos numéricos, booleanos, categóricos y fechas.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame original a optimizar.
    verbose : bool, opcional (default=True)
        Si True, muestra información sobre la reducción de memoria.
    convert_bool : bool, opcional (default=True)
        Si True, convierte columnas binarias a booleanas.
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame optimizado con tipos de datos reducidos.
    """
    import pandas as pd
    import numpy as np

    df_optimized = df.copy()
    start_mem = df_optimized.memory_usage(deep=True).sum() / 1024**2

    for col in df_optimized.columns:
        col_type = df_optimized[col].dtypes

        if pd.api.types.is_numeric_dtype(col_type):
            col_min = df_optimized[col].min()
            col_max = df_optimized[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if col_min >= 0:
                    if col_max <= np.iinfo(np.uint8).max:
                        df_optimized[col] = df_optimized[col].astype(np.uint8)
                    elif col_max <= np.iinfo(np.uint16).max:
                        df_optimized[col] = df_optimized[col].astype(np.uint16)
                    elif col_max <= np.iinfo(np.uint32).max:
                        df_optimized[col] = df_optimized[col].astype(np.uint32)
                    else:
                        df_optimized[col] = df_optimized[col].astype(np.uint64)
                else:
                    if col_min >= np.iinfo(np.int8).min and col_max <= np.iinfo(np.int8).max:
                        df_optimized[col] = df_optimized[col].astype(np.int8)
                    elif col_min >= np.iinfo(np.int16).min and col_max <= np.iinfo(np.int16).max:
                        df_optimized[col] = df_optimized[col].astype(np.int16)
                    elif col_min >= np.iinfo(np.int32).min and col_max <= np.iinfo(np.int32).max:
                        df_optimized[col] = df_optimized[col].astype(np.int32)
                    else:
                        df_optimized[col] = df_optimized[col].astype(np.int64)

            elif pd.api.types.is_float_dtype(col_type):
                if not df_optimized[col].isnull().any():
                    if col_min >= np.finfo(np.float16).min and col_max <= np.finfo(np.float16).max:
                        df_optimized[col] = df_optimized[col].astype(np.float16)
                    elif col_min >= np.finfo(np.float32).min and col_max <= np.finfo(np.float32).max:
                        df_optimized[col] = df_optimized[col].astype(np.float32)
                    else:
                        df_optimized[col] = df_optimized[col].astype(np.float64)
                else:
                    # Con NaNs, evitamos float16
                    if col_min >= np.finfo(np.float32).min and col_max <= np.finfo(np.float32).max:
                        df_optimized[col] = df_optimized[col].astype(np.float32)

        elif pd.api.types.is_object_dtype(col_type):
            num_unique = df_optimized[col].nunique()
            num_total = len(df_optimized[col])
            if num_unique / num_total < 0.5:
                df_optimized[col] = df_optimized[col].astype('category')

        elif pd.api.types.is_bool_dtype(col_type):
            df_optimized[col] = df_optimized[col].astype('bool')

        elif convert_bool and df_optimized[col].dropna().nunique() == 2:
            # Convertir columnas binarias a booleanas si no lo son aún
            unique_vals = df_optimized[col].dropna().unique()
            if set(unique_vals) <= {0, 1} or set(unique_vals) <= {True, False}:
                df_optimized[col] = df_optimized[col].astype('bool')

        elif pd.api.types.is_datetime64_any_dtype(col_type):
            # Ya está optimizada
            continue

        elif col_type == 'object':
            try:
                parsed_dates = pd.to_datetime(df_optimized[col], errors='coerce')
                if parsed_dates.notna().sum() > 0.9 * len(df_optimized[col]):
                    df_optimized[col] = parsed_dates
            except Exception:
                pass

    end_mem = df_optimized.memory_usage(deep=True).sum() / 1024**2

    if verbose:
        print(f"Memoria inicial: {start_mem:.2f} MB")
        print(f"Memoria final:   {end_mem:.2f} MB")
        print(f"Reducción:       {100 * (start_mem - end_mem) / start_mem:.2f}%")

    return df_optimized
